In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)



/home/enis/projects/nna/src


In [24]:
import numpy as np

from models_api import classicML


# Torch Model 

import torch
import exp.modelArchs




In [3]:
### example of running on a file and save it

# filename="/scratch/enis/data/nna/real/ivvavik/SINP01/2019/SINP-01_20190516_033000_vgg/SINP-01_20190516_033000_rawembeddings000.npy"
# modelPath="/home/enis/projects/nna/src/assets/sklearnModels/Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"

# model=classicML(modelPath)
# load a file and then save to correct place 
# model.classify_file(filename,"_Insect",saveAsFile=True)


In [ ]:
# ### example of running on a numpy array in memory
# import numpy as np
# results_file_path='/scratch/enis/data/nna/real/ivvavik/SINP01/2019/SINP-01_20190516_033000_Insect/SINP-01_20190516_033000_Insect000.npy'
# results=np.load(results_file_path)

# raw_embeddings = np.load(filename)
# classified = model.classify_embeddings(raw_embeddings)


In [4]:
def vectorized_y_true(humanresults,tag_set):
    y_true={tag: [None]*len(humanresults) for tag in tag_set}
    for i,tags in enumerate(humanresults.values()):
        # we  only look for tags in tag_set
        for tag in tag_set:
            if tag in tags:
                y_true[tag][i] = 1
            else:
                y_true[tag][i] = 0 
    return y_true

In [5]:
from os import listdir
from pathlib import Path
import csv

# LOAD LABELS by human
labelsbyhumanpath=Path('/scratch/enis/data/nna/labeling/resultsReal/')
# filter by username
labelsbyhuman=[i for i in listdir(labelsbyhumanpath) if (".csv" in i) ]

humanresults={}
counter=0
for apath in labelsbyhuman:
    with open(labelsbyhumanpath / apath, newline='') as f:
        reader=csv.reader(f)
        for row in reader:
            counter+=1
            humanresults[row[0]]=row[1:]

print("unique files:",len(humanresults),"\ntotal files",counter)

humanresultsAll=humanresults.copy()

unique files: 757 
total files 797


In [70]:
humanresultsSlim={}
humanresultsClipping={}
for k in humanresultsAll:
    val=humanresultsAll[k]
#     print(val)
    val2=[i.lower().replace("+","") for i in val ]
    val2=[i.lower().replace("+","") for i in val2 if i not in ["clipping","wind"]]
    if len(val2)>0:
#         print(val)
        humanresultsSlim[k]=val
    else:
        humanresultsClipping[k]=val

In [5]:
modelsPath="/home/enis/projects/nna/src/assets/sklearnModels/"
!ls /home/enis/projects/nna/src/assets/sklearnModels/

Aircraft_AdaBoost_Raw_Average_2020-03-02--14-06.joblib
Cable_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib
Insect_Linear SVM_Raw_Concat_2020-03-02--14-06.joblib
Rain_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib
Running Water_Neural Net_Raw_Concat_2020-03-02--14-06.joblib
Songbird_Neural Net_Raw_many2one_2020-03-02--14-06.joblib
Water Bird_RBF SVM_Raw_many2one_2020-03-02--14-06.joblib
Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib


In [7]:
modelList="""Aircraft_AdaBoost_Raw_Average_2020-03-02--14-06.joblib
Cable_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib
Insect_Linear SVM_Raw_Concat_2020-03-02--14-06.joblib
Rain_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib
Running Water_Neural Net_Raw_Concat_2020-03-02--14-06.joblib
Songbird_Neural Net_Raw_many2one_2020-03-02--14-06.joblib
Water Bird_RBF SVM_Raw_many2one_2020-03-02--14-06.joblib
Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"""
tagNames=[i.split("_")[0] for i in modelList.split("\n")]
modelList=[modelsPath+i for i in modelList.split("\n")]

NameError: name 'modelsPath' is not defined

In [90]:
#select the files to process
# humanresults=humanresultsAll.copy()
humanresults=humanresultsSlim.copy()
# humanresults=humanresultsClipping.copy()

print(len(humanresults))

400


In [10]:
# load data for ClassicMLModel
results={i:[] for i in tagNames}
for tagName,modelPath in zip(tagNames,modelList):
    model=classicML(modelPath,predictProbFlag=True)
    for k in humanresults:
        file_name=k.replace(".flac","_rawembed.npy")
        filepath="/scratch/enis/data/nna/labeling/split_real_embeddings/"+file_name
        raw_embeddings = np.load(filepath)
        classified = model.classify_embeddings(raw_embeddings)
        results[tagName].append(classified[0])

tagSet=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

y_true_dict=vectorized_y_true(humanresults,tagSet)




In [73]:
# humanresults

In [96]:
modelPath="/home/enis/projects/nna/src/exp/wandb/run-20200722_055604-2yqh4nns/best_model_84_ROC_AUC=0.7935.pt"
modelPath="/home/enis/projects/nna/src/exp/wandb/run-20200723_180356-18s086k5/best_model_79_ROC_AUC=0.7362.pt"
model = exp.modelArchs.NetCNN1(45,[128,850],(3,3)).float()
model.load_state_dict(torch.load(modelPath))
model.eval()


NetCNN1(
  (conv1): Conv2d(1, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1224000, out_features=10, bias=True)
  (fc1_bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc2_bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc3_bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=10, out_features=8, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)

In [97]:
import librosa
import pandas as pd

tagSet=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

y_true_dict = vectorized_y_true(humanresults,tagSet)
y_true_all = pd.DataFrame(y_true_dict)
y_true = np.array(y_true_all).astype("float")



maxLen=850
res=[]
for k in humanresults:
    filepath="/scratch/enis/data/nna/labeling/splits_real/"+k
    x,sr=librosa.load(filepath,sr=None)
    mel = librosa.feature.melspectrogram(y=x.reshape(-1), sr=44100)
    an_x = librosa.power_to_db(mel, ref=np.max)
    an_x = an_x.astype("float32")
#     if an_x.shape[1]!=938:
#         print((k,an_x.shape))
#     res.append((k,an_x.shape))
    an_x = an_x[:,:maxLen]
    x = an_x.reshape(1,1,*an_x.shape[:])
   
    out = model(torch.Tensor(x))
    out = torch.sigmoid(out)
    out = out.detach().numpy()[0]
#     print(out.value())
    res.append(out)
#     break
res=np.array(res)

In [99]:
y_true[0][5]=1

In [100]:
y_true.sum(axis=0)

array([282., 127.,  18.,  19.,  64.,   1., 208.,  28.])

In [95]:
from sklearn.metrics import roc_auc_score
score=roc_auc_score(y_true,res)
print(score)
for i,tag in enumerate(tagSet):
    print(tag)
    print(roc_auc_score(y_true[:,i],res[:,i]))

0.545026809475398
Songbird
0.3693052049525183
Water Bird
0.5565025525655447
Insect
0.7248400232693426
Running Water
0.7121149330017958
Rain
0.5638020833333333
Cable
0.03759398496240607
Wind
0.7651993189102564
Aircraft
0.6308563748079877


In [105]:
from sklearn.metrics import roc_auc_score
score=roc_auc_score(y_true,res)
print(f"{score:.2f}")
for i,tag in enumerate(tagSet):
    print(tag)
    print(f"{roc_auc_score(y_true[:,i],res[:,i]):.2f}")

0.54
Songbird
0.45
Water Bird
0.63
Insect
0.68
Running Water
0.78
Rain
0.53
Cable
0.12
Wind
0.76
Aircraft
0.34


In [88]:
# from pathlib import Path

# labelsbyhumanpath=Path('/scratch/enis/data/nna/labeling/results/')
# # splits_path= Path('/files/scratch/enis/data/nna/labeling/splits/')
# sourcePath=Path("/scratch/enis/data/nna/labeling/splits/")

    
# with open(labelsbyhumanpath/"np_array_Ymatrix.npy", 'rb') as f:
#     y_true = np.load(f)

#     # ## load Dataset
#     # # X.shape is (1300, 10, 44100)
# with open(sourcePath/"np_array_Xmatrix_shortby441000.npy", 'rb') as f:
#     X = np.load(f)
#     #
#     # ### split data
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#                 X, y_true, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#                 X_train, y_train, test_size=0.25, random_state=42)


import librosa

maxLen=850
res=[]
for x in X_val:
#     filepath="/scratch/enis/data/nna/labeling/splits_real/"+k
#     x,sr=librosa.load(filepath,sr=None)
    mel = librosa.feature.melspectrogram(y=x.reshape(-1), sr=44100)
    an_x = librosa.power_to_db(mel, ref=np.max)
    an_x = an_x.astype("float32")
#     if an_x.shape[1]!=938:
#         print((k,an_x.shape))
#     res.append((k,an_x.shape))
    an_x = an_x[:,:maxLen]
    x = an_x.reshape(1,1,*an_x.shape[:])
   
    out = model(torch.Tensor(x))
    out = torch.sigmoid(out)
    out = out.detach().numpy()[0]
#     print(out.value())
    res.append(out)
#     break
res=np.array(res)

In [89]:
from sklearn.metrics import roc_auc_score
score=roc_auc_score(y_val,res)
print(score)
for i,tag in enumerate(tagSet):
    print(tag)
    print(roc_auc_score(y_val[:,i],res[:,i]))

0.793516100966444
Songbird
0.7200205113178522
Water Bird
0.7296854521625163
Insect
0.6631027954115157
Running Water
0.923889082260123
Rain
0.6764583333333334
Cable
0.8252100840336135
Wind
0.9101562499999999
Aircraft
0.8996062992125985


In [ ]:
# 

In [11]:
from sklearn.metrics import roc_auc_score
tableResults={i:[i,] for i in tagNames}
tableResultsHeaders=["Tag","AUC","tn", "fp", "fn", "tp"]
for tagName in tagNames:
    print(tagName)
    y_true=y_true_dict[tagName]
    y_pred=results[tagName]
    if sum(y_true)==0:
        print("no positive")
        tableResults[tagName].append("NA")
        continue
    score=roc_auc_score(y_true,y_pred)
    score="{:.2}".format(score)
    print(score)
    tableResults[tagName].append(score)
    

from sklearn.metrics import confusion_matrix

for tagName in tagNames:
#     print(tagName)
    y_true=y_true_dict[tagName]
    y_pred=results[tagName]
    y_pred=np.array(y_pred)
    y_pred = np.where(y_pred > 0.5, 1, 0)
#     print(sum(y_true))
    tn, fp, fn, tp=confusion_matrix(y_true, y_pred).ravel()
    tableResults[tagName].extend([tn, fp, fn, tp])
    


Aircraft
no positive
Cable
no positive
Insect
no positive
Rain
no positive
Running Water
no positive
Songbird
no positive
Water Bird
no positive
Wind
0.56


In [12]:
from tabulate import tabulate
resultStr=tabulate(tableResults.values(), headers=tableResultsHeaders)
print(resultStr)

Tag            AUC      tn    fp    fn    tp
-------------  -----  ----  ----  ----  ----
Aircraft       NA      339    18     0     0
Cable          NA      228   129     0     0
Insect         NA      354     3     0     0
Rain           NA      354     3     0     0
Running Water  NA      329    28     0     0
Songbird       NA      194   163     0     0
Water Bird     NA      356     1     0     0
Wind           0.56     22   127     1   207


In [14]:

for k in humanresultsAll:
    val=humanresultsAll[k]
#     print(val)
    val2=[i.lower().replace("+","") for i in val ]
    val2=[i.lower().replace("+","") for i in val2 if i not in ["clipping","wind"]]
    if "aircraft" in val2:
        print("/Users/berk/Documents/research/scratch/splits_realNoClipping04/"+k)

/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10272_20190520_160000_55m_45s__55m_55s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10238_20190521_164602_33m_52s__34m_2s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10307_20190607_224602_68m_38s__68m_48s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10292_20190525_143000_46m_19s__46m_29s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10258_20190525_130000_10m_39s__10m_49s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10276_20190618_081602_46m_45s__46m_55s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10283_20190613_140000_27m_45s__27m_55s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10295_20190523_021602_41m_52s__42m_2s.flac
/Users/berk/Documents/research/scratch/splits_realNoClipping04/S4A10255_20190520_053000_57m_10s__57m_20s.flac
/Users/berk/

In [12]:

for k in humanresultsAll:
    if "S4A10295" in k:
        print(k,humanresultsAll[k])

S4A10295_20190523_021602_41m_52s__42m_2s.flac ['Songbird', 'Water Bird', 'Aircraft']
S4A10295_20190520_070000_74m_56s__75m_6s.flac ['Songbird', 'Wind']
S4A10295_20190513_211602_24m_48s__24m_58s.flac ['Songbird', 'Wind']
S4A10295_20190507_051602_15m_21s__15m_31s.flac ['Rain', 'Wind']
S4A10295_20190516_043000_41m_22s__41m_32s.flac ['Wind']
S4A10295_20190615_184602_5m_28s__5m_38s.flac ['Wind']
S4A10295_20190520_070000_20m_41s__20m_51s.flac ['Songbird', 'Wind']
S4A10295_20190615_130000_27m_25s__27m_35s.flac ['Aircraft']
S4A10295_20190605_083000_58m_49s__58m_59s.flac ['Songbird', 'Wind', 'Clipping']
S4A10295_20190601_060000_34m_22s__34m_32s.flac ['Clipping']
S4A10295_20190527_094602_37m_10s__37m_20s.flac ['Clipping+++++']


In [ ]:
# Machinery or Helicopter 
S4A10295_20190520_070000_74m_56s__75m_6s.flac ['Songbird', 'Wind'] probably wind
S4A10295_20190513_211602_24m_48s__24m_58s.flac ['Songbird', 'Wind'] wind

S4A10295_20190605_083000_58m_49s__58m_59s.flac ['Songbird', 'Wind', 'Clipping'] wind


In [ ]:
# S4A10255 automobile field
S4A10255_20190508_210000_67m_11s__67m_21s.flac ['Wind'] should have aircraft/engine
S4A10255_20190608_010000_54m_36s__54m_46s.flac ['Clipping+++'] so much like wind 
S4A10255_20190528_013000_4m_29s__4m_39s.flac ['Clipping++++'] I cannot decide, would say wind
S4A10255_20190515_044602_2m_15s__2m_25s.flac []  should probably have aircraft, frequent clipping
S4A10255_20190608_010000_54m_36s__54m_46s.flac ['Clipping+++'] so much clipping, somethong
S4A10255_20190513_140000_58m_19s__58m_29s.flac ['Wind', 'Clipping++++'] really hard to tell, in the beginning sounds like aircarft but then it sounds like wind
S4A10255_20190521_143000_14m_45s__14m_55s.flac ['Songbird', 'Clipping++++'] flight or wind 
no
no

In [ ]:
# what is flare
# how does she differentiate Commercial Airplane, Propeller Plane and Helicopter 

# maybe thats the difference of reading the image and listening